<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2d0a1f738a384c0bc4b18d9b4483efe66a07a58d8380ffb41e62485dfe676c7c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-10 09:20:59
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: 21.07 K (0.15%)
Current PnL: -28.94 L (-18.69%)
CY Booked + Current PnL: -14.65 L (-9.46%)
-------------------
Total profit:  1.14 L
Total loss:  -30.08 L
-------------------
Total Booked + Current PnL: 12.33 L (9.65%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.26%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.57 L (67.15%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.84%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.44,-17.66,21.57,0.09,17898.0,-17802.0,82975.0,92.53,35.0,M-SC,2.34,253.0,-0.99,0.62,9.01,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,2.51,-12.55,14.38,0.02,21950.0,-21915.0,152644.0,132.70,52.0,M-SC,16.63,234.0,-1.00,1.14,32.77,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.73,-2.24,24.84,22.05,23794.0,-2190.0,95790.0,-15.54,56.0,H-SC,6.87,164.0,-0.09,0.71,39.24,XR,ATH,FINANCE
43,ITC,452.00,0.32,-1.69,12.34,10.44,24279.0,-3389.0,196749.0,-41.15,45.0,X-LC,1.72,5.0,-0.14,1.46,3.78,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.66,3.09,16.59,20.19,29814.0,5393.0,179713.0,-19.08,38.0,X-MC,5.66,68.0,0.18,1.34,18.68,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,2.51,-12.55,14.38,0.02,21950.0,-21915.0,152644.0,132.70,52.0,M-SC,16.63,234.0,-1.00,1.14,32.77,OX40N,NTT,PAINTS
30,HATHWAY,31.40,1.60,-29.80,146.86,73.29,112084.0,-32400.0,76320.0,787.68,39.0,H-SC,7.35,172.0,-0.29,0.57,2.91,XR,NTT,ENTERTAINMENT
62,SATIN,274.00,1.38,-22.90,96.92,51.83,206462.0,-63261.0,213023.0,-30.35,31.0,H-SC,2.01,146.0,-0.31,1.59,3.77,XY24,NTT,FINANCE
81,VBL,671.64,1.33,-3.48,40.57,35.67,123672.0,-11006.0,304836.0,-14.99,57.0,X-LC,4.70,4.0,-0.09,2.27,9.89,X40N,ATH,FMCG
49,LAOPALA,464.00,1.32,-39.30,129.15,39.09,102776.0,-51526.0,79579.0,74.23,28.0,H-SC,7.90,155.0,-0.50,0.59,2.49,AR,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,ROUTE,2227.21,-1.55,-48.32,227.12,69.05,156181.0,-64300.0,68766.0,-58.05,47.0,H-SC,23.17,140.0,-0.41,0.51,6.05,SR,ATH,IT
65,SIEMENS,4671.50,-1.00,-16.40,50.14,25.51,78003.0,-30525.0,155570.0,-4.56,41.0,H-LC,3.88,49.0,-0.39,1.16,14.64,AR,ATH,ELECTRICAL
18,COFFEEDAY,80.00,-0.94,-46.82,154.37,35.27,93215.0,-53165.0,60384.0,-59.76,20.0,L-SC,11.88,270.0,-0.57,0.45,47.79,XR,NTT,HOTELS
79,VAIBHAVGBL,521.00,-0.91,-27.62,122.13,60.77,161561.0,-50489.0,132286.0,52.62,43.0,H-SC,4.66,159.0,-0.31,0.98,19.13,XR,NTT,JEWELLERY
48,KPIGREEN,730.66,-0.86,-20.42,84.65,46.95,84410.0,-25581.0,99716.0,-26.33,27.0,H-SC,5.75,154.0,-0.30,0.74,24.92,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.42,-7.60,125.02,107.91,170547.0,-11228.0,136416.0,-25.01,34.0,M-SC,12.21,216.0,-0.07,1.02,1.72,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,2.51,-12.55,14.38,0.02,21950.0,-21915.0,152644.0,132.70,52.0,M-SC,16.63,234.0,-1.00,1.14,32.77,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.44,-17.66,21.57,0.09,17898.0,-17802.0,82975.0,92.53,35.0,M-SC,2.34,253.0,-0.99,0.62,9.01,OX40N,NTT,DURABLES
66,SIS,528.00,0.58,-24.22,60.02,21.26,50499.0,-26895.0,84137.0,1984.97,49.0,H-SC,4.69,166.0,-0.53,0.63,13.68,OX40N,NTT,MISC
47,KANSAINER,340.00,0.43,-25.62,52.56,13.47,105422.0,-69093.0,200574.0,-69.48,28.0,H-SC,3.68,158.0,-0.66,1.49,1.99,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.55,6.16,62.27,72.27,112016.0,10439.0,179887.0,-8.8,66.0,M-LC,2.08,99.0,0.09,1.34,12.72,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.55,6.16,62.27,72.27,112016.0,10439.0,179887.0,-8.80,66.0,M-LC,2.08,99.0,0.09,1.34,12.72,XR,NTT,IT
38,INDIAMART,4810.62,0.22,-2.76,112.60,106.72,135039.0,-3408.0,119928.0,-51.77,34.0,H-SC,1.55,139.0,-0.03,0.89,19.97,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.70,-2.28,38.79,35.63,79442.0,-4773.0,204801.0,-15.85,40.0,H-MC,3.16,119.0,-0.06,1.52,14.01,AR,ATH,PHARMA
37,IEX,219.00,0.36,-3.83,54.55,48.63,104893.0,-7667.0,192287.0,-36.00,49.0,H-SC,14.06,136.0,-0.07,1.43,7.28,XR,NTT,MISC
25,FINCABLES,1641.55,0.42,-7.60,125.02,107.91,170547.0,-11228.0,136416.0,-25.01,34.0,M-SC,12.21,216.0,-0.07,1.02,1.72,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,0.03,-13.17,40.02,21.58,48700.0,-18450.0,121689.0,-56.18,24.0,X-MC,6.02,56.0,-0.38,0.91,7.73,X40N,ATH,IT
9,BAJAJHFL,181.50,0.61,-21.39,88.85,48.45,140045.0,-42886.0,157620.0,-26.96,25.0,X-MC,11.68,64.0,-0.31,1.17,2.63,X40N,ATH,FINANCE
75,TMPV,600.00,-0.09,-18.39,74.22,42.18,167427.0,-50828.0,225582.0,-26.74,25.0,X-LC,5.61,3.0,-0.30,1.68,0.00,XY24,NTT,AUTO
33,HINDUNILVR,2922.00,-0.32,-10.27,26.69,13.67,61560.0,-26413.0,230650.0,-16.94,29.0,X-LC,2.98,14.0,-0.43,1.72,8.89,XY25,NTT,FMCG
53,PAGEIND,51605.07,0.07,-9.08,38.65,26.07,57542.0,-14860.0,148880.0,-33.02,31.0,X-MC,11.83,55.0,-0.26,1.11,0.07,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,0.73,-21.56,95.68,53.50,226472.0,-65054.0,236697.0,-61.97,34.0,X-MC,1.33,58.0,-0.29,1.76,5.13,XY24,NTT,FMCG
15,CAMS,4762.0,-0.55,-80.47,538.85,24.79,265093.0,-202660.0,49196.0,-81.32,39.0,X-SC,1.54,86.0,-0.76,0.37,20.27,X40N,NTT,MISC
43,ITC,452.0,0.32,-1.69,12.34,10.44,24279.0,-3389.0,196749.0,-41.15,45.0,X-LC,1.72,5.0,-0.14,1.46,3.78,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.28,3.67,26.33,30.97,48244.0,6490.0,183228.0,-20.69,56.0,X-MC,1.84,75.0,0.13,1.36,16.69,X40,ATH,INSURANCE
3,ACC,3906.0,0.56,-24.56,117.60,64.15,211092.0,-58451.0,179500.0,-55.88,39.0,X-SC,1.95,82.0,-0.28,1.34,0.69,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TMPV,600.00,-0.09,-18.39,74.22,42.18,167427.0,-50828.0,225582.0,-26.74,25.0,X-LC,5.61,3.0,-0.30,1.68,0.00,XY24,NTT,AUTO
53,PAGEIND,51605.07,0.07,-9.08,38.65,26.07,57542.0,-14860.0,148880.0,-33.02,31.0,X-MC,11.83,55.0,-0.26,1.11,0.07,X40,ATH,APPARELS
54,PGHH,17907.65,0.06,-5.85,42.07,33.76,79544.0,-11745.0,189075.0,-33.58,34.0,X-MC,5.15,57.0,-0.15,1.41,0.14,X40,ATH,FMCG
3,ACC,3906.00,0.56,-24.56,117.60,64.15,211092.0,-58451.0,179500.0,-55.88,39.0,X-SC,1.95,82.0,-0.28,1.34,0.69,XY24,BTT,CEMENT
31,HAVELLS,2069.16,-0.31,-9.38,46.19,32.48,137292.0,-30757.0,297234.0,-13.91,40.0,X-MC,5.25,67.0,-0.22,2.21,0.98,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.19,-29.83,103.57,42.84,47241.0,-19393.0,45613.0,-54.27,41.0,X-SC,36.81,83.0,-0.41,0.34,3.20,XY24,NTT,MISC
15,CAMS,4762.00,-0.55,-80.47,538.85,24.79,265093.0,-202660.0,49196.0,-81.32,39.0,X-SC,1.54,86.0,-0.76,0.37,20.27,X40N,NTT,MISC
59,RELAXO,1176.00,-0.57,-45.80,181.00,52.31,142407.0,-66482.0,78678.0,-42.31,51.0,X-SC,6.08,91.0,-0.47,0.59,5.31,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.27,-36.34,112.39,35.20,92058.0,-46761.0,81909.0,7.44,41.0,X-SC,15.39,92.0,-0.51,0.61,4.04,X40,NTT,FOOTWEAR
22,DIXON,18931.72,0.03,-13.17,40.02,21.58,48700.0,-18450.0,121689.0,-56.18,24.0,X-MC,6.02,56.0,-0.38,0.91,7.73,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.03,-12.85,36.87,19.28,111158.0,-44470.0,301486.0,-23.95,63.0,X-LC,5.48,1.0,-0.40,2.24,11.45,X40,ATH,IT
41,INFY,2275.00,0.08,8.93,42.17,54.87,143733.0,27955.0,340843.0,-15.29,64.0,X-LC,3.31,2.0,0.19,2.54,17.96,X40,BTT,IT
75,TMPV,600.00,-0.09,-18.39,74.22,42.18,167427.0,-50828.0,225582.0,-26.74,25.0,X-LC,5.61,3.0,-0.30,1.68,0.00,XY24,NTT,AUTO
81,VBL,671.64,1.33,-3.48,40.57,35.67,123672.0,-11006.0,304836.0,-14.99,57.0,X-LC,4.70,4.0,-0.09,2.27,9.89,X40N,ATH,FMCG
43,ITC,452.00,0.32,-1.69,12.34,10.44,24279.0,-3389.0,196749.0,-41.15,45.0,X-LC,1.72,5.0,-0.14,1.46,3.78,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.58,-20.01,127.12,81.67,95388.0,-18772.0,75038.0,6738.89,31.0,L-SC,14.15,271.0,-0.20,0.56,45.31,XR,NTT,CERAMICS
50,MASFIN,398.61,0.73,-2.24,24.84,22.05,23794.0,-2190.0,95790.0,-15.54,56.0,H-SC,6.87,164.0,-0.09,0.71,39.24,XR,ATH,FINANCE
13,BSOFT,831.70,-0.07,-22.02,93.37,50.79,102404.0,-30977.0,109676.0,0.12,70.0,H-SC,5.86,151.0,-0.30,0.82,27.70,XR,ATH,IT
39,INDIGOPNTS,1408.00,2.51,-12.55,14.38,0.02,21950.0,-21915.0,152644.0,132.70,52.0,M-SC,16.63,234.0,-1.00,1.14,32.77,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.63,-1.65,22.98,20.95,56900.0,-4162.0,247606.0,-2.60,51.0,X-LC,18.97,31.0,-0.07,1.84,33.39,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.07,-22.02,93.37,50.79,102404.0,-30977.0,109676.0,0.12,70.0,H-SC,5.86,151.0,-0.30,0.82,27.70,XR,ATH,IT
32,HCLTECH,1922.01,0.41,8.01,15.48,24.73,40451.0,19387.0,261311.0,12.76,63.0,X-LC,5.97,7.0,0.48,1.94,24.47,X40,ATH,IT
84,WIPRO,420.00,0.55,6.16,62.27,72.27,112016.0,10439.0,179887.0,-8.80,66.0,M-LC,2.08,99.0,0.09,1.34,12.72,XR,NTT,IT
56,RAJESHEXPO,518.00,0.40,-63.45,173.81,0.07,87136.0,-87044.0,50133.0,1737.12,54.0,L-SC,2.29,268.0,-1.00,0.37,24.68,OX40N,NTT,JEWELLERY
39,INDIGOPNTS,1408.00,2.51,-12.55,14.38,0.02,21950.0,-21915.0,152644.0,132.70,52.0,M-SC,16.63,234.0,-1.00,1.14,32.77,OX40N,NTT,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.78
1,25,45.72
2,50,76.66


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.04
MC    30.29
LC    24.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.26
X40      22.48
X40N     10.95
XR       10.06
XY25      9.38
AR        9.32
OX40N     7.79
SR        1.03
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.49
X-MC    23.54
X-LC    21.13
M-SC    12.13
X-SC     7.04
H-MC     4.98
M-MC     1.41
L-SC     1.38
M-LC     1.34
H-LC     1.16
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.46,-7.24,41.92
IT,12.85,-16.67,79.74
FINANCE,9.53,-21.06,72.22
ELECTRICAL,6.22,-12.37,53.53
MISC,6.12,-58.01,122.74
PAINTS,6.09,-12.73,29.47
INSURANCE,4.60,-2.24,38.27
PHARMA,4.09,-1.53,34.02
AUTO,3.40,-25.51,79.32


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3238685.0,21
AR,1318396.0,10
XR,1311402.0,13
X40,1016148.0,14
X40N,978784.0,9
OX40N,746049.0,10
XY25,380772.0,6
SR,282289.0,2
MH,84410.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3685640.0,25
M-SC,1429216.0,15
X-MC,1318792.0,16
X-SC,975769.0,8
X-LC,922345.0,11
H-MC,410105.0,3
L-SC,275739.0,3
M-LC,112016.0,1
H-LC,78003.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1294380.0      6
           AR         897758.0      5
M-SC       XY24       813225.0      6
H-SC       XR         781120.0      7
X-SC       X40N       503854.0      3
X-MC       X40        460223.0      7
           XY24       395544.0      3
X-LC       X40        376521.0      5
H-SC       OX40N      345683.0      4
M-SC       OX40N      313230.0      5
X-SC       XY24       292511.0      3
H-SC       SR         282289.0      2
X-MC       X40N       276786.0      4
X-LC       XY24       247845.0      2
H-MC       AR         214925.0      2
X-LC       X40N       198144.0      2
H-MC       XY24       195180.0      1
L-SC       XR         188603.0      2
X-MC       XY25       186239.0      2
X-SC       X40        179404.0      2
M-SC       XR         175051.0      2
           AR         127710.0      2
M-LC       XR         112016.0      1
X-LC       XY25        99835.0      2
L-SC       OX40N       87136.0      1
H-SC       MH          84410.0      1
H-LC       AR          78003.0      1
M-MC       XY25        55819.0      1
L-MC       XR          54612.0      1
L-LC       XY25        38879.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
